# Benchmark Training: VGG, ResNet, ViT

Este notebook prepara datos desde los ZIPs (`reals.zip`, `fakes.zip`), carga modelos **benchmark** de torchvision (VGG, ResNet, ViT) según tu config y entrena **1 época** por transformación.

**Requisitos previos**:
- Carpeta del notebook: `notebooks/`
- Zips en `../dataset/` (al nivel del repo): `reals.zip`, `fakes.zip`
- (Opcional) modelos de usuario TorchScript en `../models/`

Si ves errores de tamaño de tensores al apilar batches, asegúrate de que los audios tengan **duración similar** o avísame para añadir padding/cropping automático.


In [2]:
# 1) Rutas y pathing del proyecto (este notebook vive en notebooks/)
import sys, os
from pathlib import Path

lib_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if lib_path not in sys.path:
    sys.path.insert(0, lib_path)
print("Project root added to sys.path:", lib_path)


Project root added to sys.path: D:\UMNG-2025\FakeVoice\FakeVoice


In [3]:
# 2) Imports principales
from pprint import pprint
from fakevoicefinder import ExperimentConfig, CreateExperiment, ModelLoader, Trainer, ConfigError


In [4]:
# 3) Configuración del experimento
cfg = ExperimentConfig()

# Nombre del experimento (carpeta bajo outputs/)
cfg.run_name = "exp_bench_v1"

# Ubicaciones (repo-relativas)
cfg.data_path = "../dataset"   # donde están reals.zip y fakes.zip
cfg.models_path = "../models"  # por si añades modelos de usuario TorchScript

# Transforms a generar (puedes dejar solo 'mel' para ir más rápido)
cfg.transform_list = ["mel"]  # ["mel", "log"]

# Modelos benchmark a probar
cfg.models_list = ["vgg16", "resnet18", "vit_b_16"]

# Entrenamiento rápido de smoke-test
cfg.type_train = "pretrain"   # 'scratch' | 'pretrain' | 'both'
cfg.epochs = 1
cfg.batch_size = 8
cfg.learning_rate = 3e-3
cfg.patience = 3

# Canal de entrada de los espectrogramas (.npy): 1 canal
cfg.input_channels = 1  # <- no está en la clase, pero ModelLoader lo respeta vía getattr(..., 3)

# Validación de la config
try:
    cfg.validate()
    print("Config validation ✅")
except ConfigError as e:
    print("Config validation error:", e)
    raise

print(cfg.summary())


Config validation ✅
ExperimentConfig:
  batch_size     : 8
  cache_features : True
  data_path      : ../dataset
  device         : gpu
  epochs         : 1
  eval_metric    : ['accuracy', 'F1']
  fakes_zip      : fakes.zip
  flag_train     : True
  input_channels : 1
  learning_rate  : 0.003
  models_list    : ['vgg16', 'resnet18', 'vit_b_16']
  models_path    : ../models
  num_workers    : 4
  optimizer      : Adam
  outputs_path   : outputs
  patience       : 3
  reals_zip      : reals.zip
  run_name       : exp_bench_v1
  save_best_only : True
  save_models    : True
  seed           : 23
  transform_list : ['mel']
  type_train     : pretrain


In [5]:
# 4) Crear experimento y preparar datos
exp = CreateExperiment(cfg, experiment_name=cfg.run_name)
exp.build()

summary = exp.prepare_data(train_ratio=0.8, seed=cfg.seed, transforms=cfg.transform_list)
print("Data prep summary:")
pprint(summary)

print("Manifest:", (exp.root / "experiment.json").as_posix())


C:\Users\Gpach\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


Data prep summary:
{'load': {'fakes': 107, 'reals': 824},
 'save_original': {'test': 187, 'train': 744},
 'split': {'test': {'fakes': 21, 'reals': 166, 'total': 187},
           'train': {'fakes': 86, 'reals': 658, 'total': 744}},
 'transforms': {'mel': {'test': 183, 'train': 656}}}
Manifest: D:/UMNG-2025/FakeVoice/FakeVoice/outputs/exp_bench_v1/experiment.json


In [7]:
# 5) Cargar y guardar modelos benchmark (loaded variants)
loader = ModelLoader(exp)
bench = loader.prepare_benchmarks(add_softmax=True, input_channels=getattr(cfg, "input_channels", 1))
print("Benchmarks saved under models/loaded:")
pprint(bench)

# User models (if any .pt/.pth under cfg.models_path)
user = loader.prepare_user_models(add_softmax=True, input_channels=cfg.input_channels)
print("User models saved:")
pprint(user)

Benchmarks saved under models/loaded:
{'resnet18': {'pretrain': 'outputs/exp_bench_v1/models/loaded/resnet18_pretrain.pt'},
 'vgg16': {'pretrain': 'outputs/exp_bench_v1/models/loaded/vgg16_pretrain.pt'},
 'vit_b_16': {'pretrain': 'outputs/exp_bench_v1/models/loaded/vit_b_16_pretrain.pt'}}
User models saved:
{'SimpleCNN_scripted.pt': 'outputs/exp_bench_v1/models/loaded/SimpleCNN_scripted_usermodel_jit.pt'}


In [8]:
def print_tree(root: Path, max_depth: int = 3, prefix: str = ""):
    if max_depth < 0:
        return
    try:
        entries = sorted(root.iterdir(), key=lambda p: (p.is_file(), p.name.lower()))
    except FileNotFoundError:
        return
    for e in entries:
        print(prefix + ("📄 " if e.is_file() else "📁 ") + e.name)
        if e.is_dir():
            print_tree(e, max_depth - 1, prefix + "   ")

print_tree(exp.root, max_depth=3)

📁 datasets
   📁 test
      📁 original
         📁 fakes
         📁 reals
      📁 transforms
         📁 mel
   📁 train
      📁 original
         📁 fakes
         📁 reals
      📁 transforms
         📁 mel
📁 models
   📁 loaded
      📄 resnet18_pretrain.pt
      📄 SimpleCNN_scripted_usermodel_jit.pt
      📄 vgg16_pretrain.pt
      📄 vit_b_16_pretrain.pt
   📁 trained
📄 experiment.json


In [14]:
# 6) Entrenar TODO (cada variante por cada transform)
trainer = Trainer(exp)
train_results = trainer.train_all()
print("Resultados de entrenamiento (rutas repo-relativas):")
pprint(train_results)

print("Best checkpoints stored in:", (exp.trained_models).as_posix())


AssertionError: Wrong image height! Expected 224 but got 128!

### Notas
- Si aparece `RuntimeError: stack expects each tensor to be equal size`, tus espectrogramas (W) tienen longitud variable.
  Podemos añadir **padding/cropping automático** en el `Trainer` o fijar una duración objetivo al transformar.
- Para probar más rápido: usa solo `mel`, baja `sample_rate` y reduce `batch_size`.
